In [43]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import subprocess
import math

In [28]:
# feature values for "/home/kai/work/data/180116ConnectC/c{}.png"
features = [[92,36,-30.5],
           [89,39,-27.7],
           [71,61,-9.25],
           [51,73,1.0],
           [83,48, -20.15],
           [101,14,-41.0],
           [60,73,0],
           [-15,37,58],
           [94,26,-34.4]]

cropBox = [-250,-300, 100, -100]

In [56]:
def getFeatureOffset(features, cropBox, inFileName, originX, originY, angle, debug=False):
    command = "{},{} {}".format(originX, originY, angle)
    outFileName = "tmp_straightened.png"
    subprocess.run(["convert", inFileName, "-distort", "SRT", command, "-alpha", "off", outFileName])

    img = cv2.imread(outFileName, cv2.IMREAD_GRAYSCALE)
    if debug:
        print(img.shape)
    img_cropped = img[originY+cropBox[1]:originY+cropBox[3], originX+cropBox[0]:originX+cropBox[2]]
    #img_cropped = img
    if debug:
        print(img_cropped.shape)

    bestVal = 100000

    for index in range(0,len(features)):
        filename = "/media/kai/Data1/robotPrototype/calibration/180116ConnectC/c{}.png".format(index)
        cTemplate = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        w, h = cTemplate.shape[::-1]

        result = cv2.matchTemplate(img_cropped, cTemplate, cv2.TM_SQDIFF_NORMED)
        (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)
        if debug:
            print(index, minVal)

        if minVal < bestVal:
            bestVal = minVal
            bestLoc = minLoc
            bestIndex = index
            bestResult = result

    top_left = bestLoc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    if debug:
        print("best match: ", bestIndex, top_left, bottom_right)
    resultX = cropBox[0]+top_left[0]+features[bestIndex][0]
    resultY = cropBox[1]+top_left[1]+features[bestIndex][1]
    resultAngle = features[bestIndex][2]
    
    # return pose relative to origin
    resultX = -0.0001618 * resultX
    resultY = -0.0001627 * resultY
    resultZ = -0.005
    rX = 0.139
    rY = -0.35
    rZ = resultAngle * math.pi/180.0
    if debug:
        print("result (pixels offset from origin): ({}, {}) , angle = {}".format(resultX, resultY, resultAngle))
    
    if debug:
        filename = "/home/kai/work/data/180116ConnectC/c{}.png".format(bestIndex)
        cTemplate = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        cv2.rectangle(img_cropped,top_left, bottom_right, 255, 3)
        plt.subplot(121),plt.imshow(img_cropped,cmap = 'gray')
        plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
        plt.subplot(122),plt.imshow(cTemplate,cmap = 'gray')
        plt.title('template image'), plt.xticks([]), plt.yticks([])
        plt.show()
        
    return "({}, {}, {}, {}, {}, {})\n".format(resultX, resultY, resultZ, rX, rY, rZ)

In [57]:
inFileName = '/media/kai/Data1/robotPrototype/data/180116SearchOriginAndPointCloud/undistorted.png'
#inFileName = '/home/kai/work/data/180116ConnectC/img11.jpg'


#rotate this around origin to reference plane
# (620.5 447.0), angle = 5.5309909440
originX = round(620.5)
originY = 447
angle = -5.531

result = getFeatureOffset(features, cropBox, inFileName, originX, originY, angle, debug=False)
print(result)

(0.009708, 0.0383972, -0.005, 0.139, -0.35, -0.6003932626860493)

